In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm

/home/ogech/CPC/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Possible next steps**

- Potential datasets; R22, RF fingerprinting and RF class dataset

### **CPC model development**

In [2]:
class CPC(nn.Module):
    def __init__(self,
                 in_features: int,
                 hidden_features: int,
                 slice_length: int = 1024,
                 history_steps: int = 12,
                 drop_rate: float = 0.2,
                 drop_path_rate: float = 0.7):
        super(CPC, self).__init__()

        # define some hyperparameters
        self.slice_length = slice_length
        self.history_steps = history_steps

        # define an encoder
        self.encoder = timm.create_model(
            "resnet18",
            in_chans=in_features,
            num_classes=hidden_features,
            drop_rate=drop_rate,
            drop_path_rate=drop_path_rate,
        )

        # define the autoregressive model
        self.autoregressor = nn.GRU(
            input_size=hidden_features,
            hidden_size=hidden_features,
            num_layers=1,
            batch_first=True
        )

    def forward(self, x: torch.Tensor):
        # x is of shape (B, Ca, C, T)

        # some preprocessing: slice the input into chunks of length slice_length w/o overlap
        x = x.unfold(-1, self.slice_length,
                     self.slice_length).permute(0, 3, 1, 2, 4)  # (B, num_chunks, Ca, C, slice_length)

        # pass the input through the encoder
        B, N, Ca, C, T = x.size()
        x = self.encoder(x.contiguous().view(B*N, Ca, C, T))
        x = x.view(B, N, -1)  # (B, num_chunks, D)

        # pass the history_steps chunks through the autoregressive model
        h0 = torch.zeros(1, x.size(0), x.size(-1)
                         ).to(x.device)  # (num_layers, B, D)

        # c, h0 = torch.stack(
        #     [self.autoregressor(x[:, :t])
        #                         for t in range(1, self.history_steps + 1)]
        #     )  # (B, history_steps, D) # noqa

        # return c[:, -1], x[:, self.history_steps:], h0

        ct = torch.zeros(B, self.history_steps, x.size(-1)).to(x.device)
        preds = torch.zeros(B, self.history_steps, x.size(-1)).to(x.device)
        for t in range(1, self.history_steps + 1):
            out, h0 = self.autoregressor(x[:, :t], h0)
            ct[:, t-1, :] = out[:, -1, :]
            preds[:, t-1, :] = x[:, t, :]

        return ct, preds, h0

In [3]:
def nt_xent_loss(c_t, z_fut, temperature=1.0):
    # c_t and z_fut is of shape (B, K, D)
    # normalize the vectors
    c_t = F.normalize(c_t, p=2, dim=-1)
    z_fut = F.normalize(z_fut, p=2, dim=-1)

    B, K, D = z_fut.size()
    # compute the cosine similarity
    logits = torch.einsum("bjd, bkd -> bjk", c_t, z_fut) / \
        temperature  # (B, K, K)

    # plot the logits
    # import matplotlib.pyplot as plt
    # plt.imshow(logits.view(B*K, K).detach().cpu().numpy(),
    #            cmap='hot', interpolation='nearest', aspect='auto')
    # positives are the diagonal pairs j == k
    true_labels = torch.arange(K).repeat(B).to(c_t.device)  # (B*K,)

    # compute the loss
    return F.cross_entropy(logits.view(B*K, K), true_labels)

In [4]:
from torchvision.datasets import DatasetFolder
from torch.utils.data import DataLoader
import numpy as np
from tqdm.autonotebook import tqdm
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import os
import shutil
import yaml
import pandas as pd
from sklearn.model_selection import train_test_split
import copy

In [5]:
config = {
    "in_features": 4,
    "hidden_features": 256,
    "slice_length": 1024,
    "history_steps": 8,
    "drop_rate": 0.0,
    "drop_path_rate": 0.0,
    "temperature": 0.1,
    "batch_size": 128,
    "experiment_name": f"cpc_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}",
    "num_epochs": 100,
    "knn_k": 200,
    "knn_t": 0.1,
    "learning_rate": 1.8e-3,
    "weight_decay": 2.5e-6,
    "dataset_path": "./oct10_outdoor_gain_experiments",
    "checkpoint_path": None,
    "comment": "CPC model training",
}

### **Data prep**

In [ ]:
# split the dataset into train and test
records = []
for dirpath, _, files in os.walk(config["dataset_path"]):
    for file in files:
        filepath = os.path.join(dirpath, file)
        with open(filepath, "rb") as f:
            _ = np.load(f)
            metadata = np.load(f, allow_pickle=True).item()
            mod, *_, gain = dirpath.split("/")[-1].split("_")
            # strip g from the gain
            gain = int(gain[1:])
            records.append({
                "filepath": filepath,
                "azimuth": metadata["servo_azimuth"],
                "elevation": metadata["servo_elevation"],
                "mod": mod,
                "gain": gain,
            })

In [ ]:
df = pd.DataFrame(records)
df["strata"] = list(zip(df["azimuth"], df["elevation"]))
# df["strata"].value_counts().sort_values(ascending=False)

train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df["strata"],
)

In [ ]:
train_df.drop("strata", axis=1).to_csv("train.csv", index=False)
test_df.drop("strata", axis=1).to_csv("test.csv", index=False)

In [ ]:
# move files to the train and test folders
os.makedirs("datasets/train", exist_ok=True)
os.makedirs("datasets/test", exist_ok=True)
for _, row in train_df.iterrows():
    os.makedirs(os.path.join("datasets/train", row["mod"]), exist_ok=True)
    shutil.move(row["filepath"], os.path.join("datasets/train", row["mod"]))
for _, row in test_df.iterrows():
    os.makedirs(os.path.join("datasets/test", row["mod"]), exist_ok=True)
    shutil.move(row["filepath"], os.path.join("datasets/test", row["mod"]))

# delete the original dataset folder
shutil.rmtree(config["dataset_path"])

### **Training**

In [6]:
class R22_Dataset(DatasetFolder):
    """
    Ensures every sample tensor ends up the same length (the minimum
    length across your entire dataset), by trimming.
    """

    def __init__(self, root, load_fn, transform=None, **kwargs):
        super().__init__(root, loader=load_fn, transform=transform, **kwargs)
        # # 1) scan every file once to find the minimum length
        # lengths = []
        # for path, _ in self.samples:
        #     # load only the array header, not the entire payload
        #     arr = np.load(path, mmap_mode='r', allow_pickle=False)
        #     lengths.append(arr.shape[-1])
        # self.min_length = min(lengths)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        path = self.samples[index][0]
        sample, target = self.loader(path)
        if self.transform is not None:
            sample = self.transform(sample)
        # # 2) trim the sample to the minimum length
        # sample = sample[..., :self.min_length]
        return sample, target


with open("label_encoder_angles.yaml", "r") as f:
    label_encoder_angles = yaml.safe_load(f)

class2idx = {c: i for i, c in enumerate(label_encoder_angles["classes"])}


def load_fn(path):
    with open(path, "rb") as f:
        data = np.load(f)
        metadata = np.load(f, allow_pickle=True).item()
    data = np.stack((data.real, data.imag), axis=1)
    # drop the first 1024 samples
    data = data[:, :, 1024:]
    # drop the last 1024 samples
    data = data[:, :, :-1024]
    # CPC uses a portion of the entire sequence in the loss
    min_sequence_length = int(
        config["slice_length"] * (config["history_steps"] + 1))
    idx = np.random.randint(0, data.shape[-1] - min_sequence_length)
    data = data[:, :,  idx:idx + min_sequence_length]
    # # normalize the data
    # data = (data - np.mean(data, axis=(1, 2), keepdims=True)) / \
    #     (np.std(data, axis=(1, 2), keepdims=True) + 1e-8)
    return data.astype(np.float32), class2idx.get(
        str(tuple((metadata["servo_azimuth"], metadata["servo_elevation"]))))


train_ds = R22_Dataset(
    root="./datasets/train",
    load_fn=load_fn,
    transform=lambda x: torch.from_numpy(x),
    extensions=[".npy"],
)

test_ds = R22_Dataset(
    root="./datasets/test",
    load_fn=load_fn,
    transform=lambda x: torch.from_numpy(x),
    extensions=[".npy"],
)

train_ds[0][0].shape, train_ds[0][1], test_ds[0][0].shape, test_ds[0][1]

(torch.Size([4, 2, 9216]), 96, torch.Size([4, 2, 9216]), 93)

In [8]:
train_loader = DataLoader(
    train_ds,
    batch_size=config["batch_size"],
    shuffle=True,
    num_workers=8,  # Adjust based on your system
    pin_memory=True,
    prefetch_factor=2,
    persistent_workers=True,
)

memory_loader = DataLoader(
    train_ds,
    batch_size=config["batch_size"],
    shuffle=False,
    num_workers=8,  # Adjust based on your system
    pin_memory=True,
    prefetch_factor=2,
    persistent_workers=True,
)

test_loader = DataLoader(
    test_ds,
    batch_size=config["batch_size"],
    shuffle=False,
    num_workers=8,  # Adjust based on your system
    pin_memory=True,
    prefetch_factor=2,
    persistent_workers=True,
)

xb, yb = next(iter(train_loader))
print(xb.shape, yb.shape)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.Size([128, 4, 2, 9216]) torch.Size([128])


In [9]:
model = CPC(
    in_features=config["in_features"],
    hidden_features=config["hidden_features"],
    slice_length=config["slice_length"],
    history_steps=config["history_steps"],
    drop_rate=config["drop_rate"],
    drop_path_rate=config["drop_path_rate"],
).to(device)
c_t, z_fut, _ = model(xb.to(device))
print(c_t.shape, z_fut.shape)
loss = nt_xent_loss(c_t, z_fut)
print(loss)

torch.Size([128, 8, 256]) torch.Size([128, 8, 256])
tensor(2.0795, device='cuda:0', grad_fn=<NllLossBackward0>)


In [10]:
class Tracker:
    """
    A class to track the best value of a metric.

    :param metric: The name of the metric to track. If 'loss' is in the metric name, the goal is to minimize it.
    :type metric: str
    :param mode: The mode of tracking. Can be 'auto', 'min', or 'max'. Default is 'auto'.
    :type mode: str, optional
    """

    def __init__(self, metric, mode='auto'):
        self.metric = metric
        self.mode = mode
        self.mode_dict = {
            'auto': np.less if 'loss' in metric else np.greater,
            'min': np.less,
            'max': np.greater
        }
        self.operator = self.mode_dict[mode]
        self._best = np.inf if self.operator == np.less else -np.inf

    @property
    def best(self):
        return self._best

    @best.setter
    def best(self, value):
        self._best = value

In [11]:
def knn_predict(feature, feature_bank, feature_labels, num_classes, k=200, t=0.1):
    # feature is [b, d], feature_bank is [d, n] and feature_labels is [n]
    sim_mat = torch.mm(feature, feature_bank)  # [b, n]
    sim_weight, sim_indices = sim_mat.topk(k, dim=-1)  # [b, k]
    sim_labels = torch.gather(feature_labels.expand(
        feature.size(0), -1), dim=-1, index=sim_indices)
    sim_weight = (sim_weight / t).exp()

    # count for each class
    one_hot = torch.zeros(feature.size(0) * k, num_classes).to(feature.device)
    one_hot = one_hot.scatter(
        dim=-1, index=sim_labels.view(-1, 1), value=1.0)  # [b*k, num_classes]
    pred_scores = torch.sum(one_hot.view(feature.size(0), -1, num_classes) * sim_weight.unsqueeze(dim=-1), dim=1)  # weighted scores [b, num_classes] # noqa

    pred_labels = pred_scores.argsort(
        dim=-1, descending=True)  # [b, num_classes]
    return pred_labels


@torch.no_grad()
def knn_evaluate(model, memory_loader, test_loader, epoch, config, pbar, writer, device):
    feature_bank, feature_labels = [], []
    encoder = copy.deepcopy(model.encoder)
    encoder.fc = nn.Identity()
    encoder = encoder.to(device)
    encoder.eval()
    for x, y in memory_loader:
        x, y = x.to(device), y.to(device)
        feature = encoder(x)
        # normalize the feature
        feature = F.normalize(feature, dim=-1)
        feature_bank.append(feature)
        feature_labels.append(y)
    feature_bank = torch.cat(feature_bank, dim=0).t().contiguous()  # [d, n]
    feature_labels = torch.cat(feature_labels, dim=0)  # [n]

    # loop over the test set
    total_num, top1, top5 = 0, 0, 0
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        feature = encoder(x)
        # normalize the feature
        feature = F.normalize(feature, dim=-1)
        pred_labels = knn_predict(feature, feature_bank, feature_labels,
                                  num_classes=config["nclasses"], k=config["knn_k"], t=config["knn_t"])
        top1 += (pred_labels[:, 0] == y).sum().item()
        top5 += (pred_labels[:, :5] == y.unsqueeze(1)).sum().item()
        total_num += y.size(0)
    pbar.write(
        f"Epoch [{epoch}/{config['num_epochs']}] Acc@1: {top1 / total_num * 100:.2f}%, Acc@5: {top5 / total_num * 100:.2f}%"
    )
    writer.add_scalar(
        f"test/top1", top1 / total_num * 100, epoch)
    writer.add_scalar(
        f"test/top5", top5 / total_num * 100, epoch)

In [12]:
# add the label encoder angles to the config
config.update(**label_encoder_angles)
# dump the config
path = os.path.join("./experiments", config["experiment_name"])
os.makedirs(path, exist_ok=True)
with open(os.path.join(path, "config.yaml"), "w") as f:
    yaml.dump(config, f, default_flow_style=False)

optim = torch.optim.AdamW(
    model.parameters(),
    lr=config["learning_rate"],
    weight_decay=config["weight_decay"],
)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optim,
    T_max=config["num_epochs"],
    eta_min=1e-6,
)

scaler = torch.amp.GradScaler(device=device.type, enabled=True)
writer = SummaryWriter(
    log_dir=os.path.join("./experiments/", config["experiment_name"]),
)
tracker = Tracker("loss/epoch", mode="min")

start_epoch = 0
# resume training from a checkpoint if it exists
checkpoint_path = config["checkpoint_path"]
if checkpoint_path and os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint["state_dict"])
    optim.load_state_dict(checkpoint["opt_state_dict"])
    scheduler.load_state_dict(checkpoint["sch_state_dict"])
    start_epoch = checkpoint["epoch"]
    print(f"Resuming training from epoch {start_epoch}")

step = 0
with tqdm(range(start_epoch, config["num_epochs"])) as master_bar:
    for epoch in master_bar:
        model.train()
        avg_loss, total_num, top1, feature_bank = 0.0, 0, 0, []
        with tqdm(train_loader) as pbar:
            for xb, yb in pbar:
                xb = xb.to(device)
                optim.zero_grad()

                with torch.amp.autocast(device_type=device.type,
                                        enabled=True):
                    c_t, z_fut, _ = model(xb)
                    loss = nt_xent_loss(c_t, z_fut)

                scaler.scale(loss).backward()
                # clip gradients
                scaler.unscale_(optim)
                norm = torch.nn.utils.clip_grad_norm_(
                    model.parameters(), 1.0, norm_type=2
                )
                scaler.step(optim)
                scaler.update()

                avg_loss += loss.item()

                pbar.set_postfix(
                    {"loss/step": loss.item(), "norm": norm.item()})
                writer.add_scalar("loss/step", loss.item(), step)
                writer.add_scalar("norm/step", norm.item(), step)
                step += 1

        avg_loss /= len(train_loader)
        writer.add_scalar("loss/epoch", avg_loss, epoch)
        master_bar.write(f"Epoch {epoch}: loss = {avg_loss:.4f}")
        scheduler.step()
        writer.add_scalar(
            "learning_rate/epoch",
            optim.param_groups[0]["lr"],
            epoch,
        )

        # evaluate the model
        knn_evaluate(
            model,
            memory_loader,
            test_loader,
            epoch,
            config,
            master_bar,
            writer,
            device,
        )

        writer.flush()
        if tracker.operator(avg_loss, tracker.best):
            tracker.best = avg_loss
            # Save the model checkpoint
            checkpoint_path = os.path.join(
                "./experiments", f"{config['experiment_name']}/weights.pth"
            )
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Model saved to {checkpoint_path}")
        # save the latest checkpoint
        checkpoint_path = os.path.join(
            "./experiments", f"{config['experiment_name']}/last_checkpoint.pt")
        torch.save({
            "state_dict": model.state_dict(),
            "opt_state_dict": optim.state_dict(),
            "sch_state_dict": scheduler.state_dict(),
            "epoch": epoch}, checkpoint_path)
        print(
            f"Latest checkpoint saved to {checkpoint_path} at epoch {epoch}"
        )

writer.close()
print("Training complete.")

  0%|          | 0/100 [01:39<?, ?it/s]

Epoch 0: loss = 2.0761


  0%|          | 0/100 [03:19<?, ?it/s]

Epoch [0/100] Acc@1: 0.65%, Acc@5: 6.29%
Model saved to ./experiments/cpc_2025-05-14_06-46-08/weights.pth


  1%|          | 1/100 [03:19<5:29:29, 199.69s/it]

...

Epoch 99: loss = 1.5009


 99%|█████████▉| 99/100 [4:27:36<02:41, 161.00s/it]

Epoch [99/100] Acc@1: 0.00%, Acc@5: 0.11%


100%|██████████| 100/100 [4:27:36<00:00, 160.57s/it]

Latest checkpoint saved to ./experiments/cpc_2025-05-14_06-46-08/last_checkpoint.pt at epoch 99
Training complete.


##### **Notes**

- training has very dramatic gradient that dances around and loss is very noisy
- use of gradient clipping and data normalization per sample (try whole batch normalization too) reduced the noisy loss but more improvements can be done
- TODO: introduce the silhoutte score to determining if the clustering is improving during training

##### **Updates on 07/05/2025**
- changes to CPC model and loss based on training implement discussed in paper resolve most of the problem oberseved before

##### **Updates on 12/05/2025**

- use of more negative sampling in training dataset

#### **Updates on 14/05/2025**

- use of knn evaluation to track clustering (indication of learning but very slow compared to SimCLR and MoCo)

### **Linear-probing**

In [ ]:
from torch.utils.data import Dataset
import h5py
from model_builder.core import Learner, DataLoaders, SaveModelCallback
from model_builder.core.metrics import multiclass_accuracy
from datetime import datetime
import yaml
import os

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
config = {
    "num_epochs": 5,
    "lr": 1e-3,
    "weight_decay": 1e-4,
    "bs": 512,
    "drop_rate": 0.2,
    "drop_path_rate": 0.7,
    "experiment_name": f"cpc_linear_probing_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}",
    "pretrained_backbone": "./experiments/cpc_2025-05-12_15-50-11/weights.pth",
}

with open("label_encoder_angles.yaml", "r") as f:
    label_encoder = yaml.safe_load(f)
    config.update(**label_encoder)

config

In [ ]:
# load the best model
model = timm.create_model(
    "resnet18",
    in_chans=4,
    num_classes=225,
    drop_rate=config["drop_rate"],
    drop_path_rate=config["drop_path_rate"],
)

linear_name = "fc"

weights = torch.load(
    config["pretrained_backbone"], weights_only=True, map_location=device
)

# clean out ssl base encoder weights
for k in list(weights.keys()):
    # retain only encoder up to before the embedding layer
    if k.startswith('encoder') and not k.startswith('encoder.%s' % linear_name):
        # remove prefix
        weights[k[len("encoder."):]] = weights[k]
    # delete renamed or unused k
    del weights[k]

msg = model.load_state_dict(
    weights, strict=False)
assert msg.missing_keys == \
    [f"{linear_name}.weight", f"{linear_name}.bias"], \
    "Missing keys: %s" % msg.missing_keys

in_features = model.fc.in_features
model.fc = nn.Identity()

In [ ]:
class R22_Dataset(Dataset):
    """
    Avoid loading the entire dataset into memory.
    """

    def __init__(self, h5_path,
                 input_label="iq_data",
                 target_label="angles",
                 transform=None):
        self.h5_path = h5_path
        self.input_label = input_label
        self.target_label = target_label
        self.transform = transform
        self._file = None

    def _ensure_open(self):
        if self._file is None:
            # open in read-only, latest libver, SWMR if you know no writers
            self._file = h5py.File(
                self.h5_path, 'r', libver='latest', swmr=True)
            self._x = self._file[self.input_label]
            self._y = self._file[self.target_label]

    def __len__(self):
        self._ensure_open()
        return self._x.shape[0]

    def __getitem__(self, idx):
        self._ensure_open()
        x = self._x[idx]
        y = self._y[idx]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __del__(self):
        if self._file is not None:
            self._file.close()

In [ ]:
# training config
bs = config["bs"]
prefetch_factor = 2
num_workers = 8
persistent_workers = True
pin_memory = True


# create a dataset
train_ds = R22_Dataset(
    "datasets/train_preprocessed.h5",
    transform=lambda x: torch.from_numpy(x).float()
)

print(f"train_ds: {len(train_ds):,}")

val_ds = R22_Dataset(
    "datasets/test_preprocessed.h5",
    transform=lambda x: torch.from_numpy(x).float()
)
print(f"val_ds: {len(val_ds):,}")

dls = DataLoaders.create(
    train_ds,
    val_ds,
    bs=bs,
    prefetch_factor=prefetch_factor,
    num_workers=num_workers,
    persistent_workers=persistent_workers,
    pin_memory=pin_memory,
)

In [ ]:
import re


def sanity_check(state_dict, pretrained_backbone, linear_name):
    """
    check that the pretrained weights (especially the BN layers) are not updated in the linear probing step
    """
    print("=> loading '{}' for sanity check".format(pretrained_backbone))
    state_dict_pre = torch.load(
        pretrained_backbone, weights_only=True, map_location="cpu"
    )

    # define pattern to ignore linear layers
    pattern = r'%s(\.weight|\.[0-9]+\.weight|\.bias|\.[0-9]+\.bias)' % re.escape(
        linear_name)
    for k in list(state_dict.keys()):
        if re.search(pattern, k):
            continue

        k_pre = k

        assert ((state_dict[k].cpu() == state_dict_pre[k_pre]).all()), \
            "{} is changed in linear probing training".format(k)
    print("=> sanity check passed")

In [ ]:
from copy import deepcopy


class LinearClassifier(nn.Module):
    def __init__(self, encoder: nn.Module,
                 linear_name: str,
                 feature_dim: int,
                 num_classes: int):
        super(LinearClassifier, self).__init__()
        self.encoder = deepcopy(encoder)
        setattr(self.encoder, linear_name, nn.Identity())
        self.fc = nn.Linear(feature_dim, num_classes)

    def forward(self, x: torch.Tensor):
        with torch.no_grad():
            x = self.encoder(x)
        x = self.fc(x)
        return x

In [ ]:
model = LinearClassifier(
    encoder=model,
    linear_name=linear_name,
    feature_dim=in_features,
    num_classes=config["nclasses"],
)

In [ ]:
learner_probing = Learner(
    dls=dls,
    model=model,
    opt_func=torch.optim.Adam,
    loss_func=torch.nn.CrossEntropyLoss(),
    metrics=[multiclass_accuracy],
    model_dir=os.path.join(
        "./experiments", f"{config['experiment_name']}"),
).to_fp16()

In [ ]:
learner_probing.lr_find(start_lr=1e-5, num_it=1000)
learner_probing.recorder.plot(suggestion=True, show_grid=True)

In [ ]:
sanity_check(
    model.state_dict(), config["pretrained_backbone"], linear_name,
)

In [ ]:
learner_probing.fit_fc(
    5,
    curve_type="cosine",
    lr=5.68e-03,
    cbs=[SaveModelCallback(learner_probing, "val_loss",
                           name="resnet18_w_frozen_cpc")],
)

In [ ]:
learner_probing.recorder.plot_results()

In [ ]:
loss, acc = learner_probing.validate(plot_cm=True)